In [1]:
import os
import pandas as pd
import codecs
from io import StringIO

types_of_encoding = ["utf-8", "cp1252", "cp850", "latin1", "utf8"]
SCORE_DIR = "data/scores"

In [2]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [3]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    for encType in types_of_encoding:
        try:
            with open(box_score, encoding=encType) as f:
                html = f.read()
        
            soup = BeautifulSoup(html)
            [s.decompose() for s in soup.select("tr.over_header")]
            [s.decompose() for s in soup.select("tr.thead")]
        except UnicodeDecodeError:
            pass
    return soup

In [4]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [5]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [6]:
def read_stats(soup, team, stat):
    df = pd.read_html(StringIO(str(soup)), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [7]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 11520
200 / 11520
300 / 11520
400 / 11520
500 / 11520
600 / 11520
700 / 11520
800 / 11520
900 / 11520
1000 / 11520
1100 / 11520
1200 / 11520
1300 / 11520
1400 / 11520
1500 / 11520
1600 / 11520
1700 / 11520
1800 / 11520
1900 / 11520
2000 / 11520
2100 / 11520
2200 / 11520
2300 / 11520
2400 / 11520
2500 / 11520
2600 / 11520
2700 / 11520
2800 / 11520
2900 / 11520
3000 / 11520
3100 / 11520
3200 / 11520
3300 / 11520
3400 / 11520
3500 / 11520
3600 / 11520
3700 / 11520
3800 / 11520
3900 / 11520
4000 / 11520
4100 / 11520
4200 / 11520
4300 / 11520
4400 / 11520
4500 / 11520
4600 / 11520
4700 / 11520
4800 / 11520
4900 / 11520
5000 / 11520
5100 / 11520
5200 / 11520
5300 / 11520
5400 / 11520
5500 / 11520
5600 / 11520
5700 / 11520
5800 / 11520
5900 / 11520
6000 / 11520
6100 / 11520
6200 / 11520
6300 / 11520
6400 / 11520
6500 / 11520
6600 / 11520
6700 / 11520
6800 / 11520
6900 / 11520
7000 / 11520
7100 / 11520
7200 / 11520
7300 / 11520
7400 / 11520
7500 / 11520
7600 / 11520
7700 / 11520
7800 / 1

In [8]:
games_df = pd.concat(games, ignore_index=True)

In [9]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,240.0,240.0,38.0,86.0,0.442,9.0,25.0,0.360,14.0,16.0,...,50.0,35.7,205.0,115.0,BOS,106,0,2024,2024-06-12,False
23036,240.0,240.0,29.0,80.0,0.363,14.0,41.0,0.341,12.0,13.0,...,100.0,40.8,212.0,99.0,DAL,122,1,2024,2024-06-14,False
23037,240.0,240.0,46.0,91.0,0.505,15.0,37.0,0.405,15.0,22.0,...,50.0,35.9,200.0,138.0,BOS,84,0,2024,2024-06-14,True
23038,240.0,240.0,35.0,78.0,0.449,11.0,37.0,0.297,7.0,13.0,...,15.5,32.3,300.0,110.0,BOS,106,1,2024,2024-06-17,False


In [10]:
games_df.to_csv("C:\\Users\\yb200\\PycharmProjects\\nbaML\\nba_games.csv")